In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

In [ ]:
""" Model """
 
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import transforms, models
 
from PIL import Image
import matplotlib.pyplot as plt
 
import copy
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
imsize = 512 if torch.cuda.is_available() else 128
 
loader = transforms.Compose([transforms.Resize(imsize),
                             transforms.ToTensor()])
unloader = transforms.ToPILImage()
 
class ContentLoss(nn.Module):
 
  def __init__(self, target):
    super().__init__()
 
    self.target = target.detach()
 
  def forward(self, input):
    
    self.loss = F.mse_loss(input, self.target)
 
    return input
 
def gram_matrix(input):
 
  a, b, c, d = input.size()
  features = input.view(a * b, c * d)
  G = torch.mm(features, features.t())
 
  return G.div(a * b * c * d)
 
class StyleLoss(nn.Module):
 
  def __init__(self, target_features):
    super().__init__()
 
    self.target = gram_matrix(target_features).detach()
  
  def forward(self, input):
 
    G = gram_matrix(input)
    self.loss = F.mse_loss(G, self.target)
 
    return input
 
cnn = models.vgg19(pretrained=True).features.to(device).eval()
 
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)
 
class Normalization(nn.Module):
  
  def __init__(self, mean, std):
    super().__init__()
 
    self.mean = torch.tensor(mean).view(-1, 1, 1)
    self.std = torch.tensor(std).view(-1, 1, 1)
  
  def forward(self, img):
 
    return (img - self.mean) / self.std
 
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4']
 
def get_style_model_and_losses(
    cnn, normalization_mean, normalization_std,
    style_img, content_img,
    content_layers=content_layers_default,
    style_layers=style_layers_default):
  
  cnn = copy.deepcopy(cnn)
 
  normalization = Normalization(normalization_mean, normalization_std)\
  .to(device)
 
  content_losses = []
  style_losses = []
 
  model = nn.Sequential(normalization)
 
  i = 0
  for layer in cnn.children():
 
    if isinstance(layer, nn.Conv2d):
 
      i += 1
      name = 'conv_{}'.format(i)
    
    elif isinstance(layer, nn.ReLU):
 
      name = 'relu_{}'.format(i)
      layer = nn.ReLU(inplace=False)
    
    elif isinstance(layer, nn.MaxPool2d):
 
      name = 'pool_{}'.format(i)
    
    elif isinstance(layer, nn.BatchNorm2d):
 
      name = 'bn_{}'.format(i)
    
    else:
 
      raise RuntimeError('Unrecognized layer: {}'\
                         .format(layer.__class__.__name__))
 
    model.add_module(name, layer)
 
    if name in content_layers:
 
      target = model(content_img).detach()
      content_loss = ContentLoss(target)
      model.add_module('content_loss_{}'.format(i), content_loss)
      content_losses.append(content_loss)
    
    if name in style_layers:
 
      target_feature = model(style_img).detach()
      style_loss = StyleLoss(target_feature)
      model.add_module('style_loss_{}'.format(i), style_loss)
      style_losses.append(style_loss)
 
  for i in range(len(model) - 1, -1, -1):
 
    if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
      break
 
  return model, style_losses, content_losses  
 
def get_input_optimizer(input_img):
 
  optimizer = optim.LBFGS([input_img.requires_grad_()])
 
  return optimizer
 
def run_style_transfer(
    cnn, normalization_mean, normalization_std,
    content_img, style_img, input_img, num_steps=300,
    style_weight=1000000, content_weight=1):
  
  print('Building the style transfer model...')
  model, style_losses, content_losses = get_style_model_and_losses(
      cnn, normalization_mean, normalization_std,
      style_img, content_img
  )
 
  print('Optimizing...')
  optimizer = get_input_optimizer(input_img)
 
  run = [0]
  while run[0] <= num_steps:
 
    def closure():
 
      input_img.data.clamp_(0, 1)
 
      optimizer.zero_grad()
 
      model(input_img)
      style_score = 0
      content_score = 0
 
      for sl in style_losses:
 
        style_score += sl.loss
      
      for cl in content_losses:
 
        content_score += cl.loss
      
      style_score *= style_weight
      content_score *= content_weight
 
      loss = style_score + content_score
      loss.backward()
 
      run[0] += 1
      if run[0]%50 == 0:
 
        print('run {}'.format(run))
        print('Syle Loss: {:.4f} Content Loss: {:.4f}'.format(
            style_score.item(), content_score.item()
        ))
        print()
      
      return style_score + content_score
    
    optimizer.step(closure)
  
  input_img.data.clamp_(0, 1)
 
  return input_img

In [ ]:
""" Preprocessing """

def make_square(im, min_size=imsize, fill_color=(255, 255, 255)):
    x, y = im.size
    size = max(min(x, y), min_size)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

def load_from_pil(img):
  image = loader(img).unsqueeze(0)
  return image.to(device, torch.float)
 
def preprocess(content, style):
 
  content = Image.open(content)
  style = Image.open(style)
 
  content, style = make_square(content), make_square(style)
 
  content, style = load_from_pil(content), load_from_pil(style)
 
  return content, style

In [ ]:
""" Model runner """
 
def run(content, style):
 
  content_image, style_image = preprocess(content, style)
 
  input_image = content_image.clone()
 
  generated_image = run_style_transfer(cnn, 
                                       cnn_normalization_mean, cnn_normalization_std, 
                                       content_image, style_image, input_image)
  image = generated_image.cpu().clone()
  image = image.squeeze(0)
  image = unloader(image)
 
  image.save('generated.jpg')

In [ ]:
""" Get frontend files """
 
!mkdir templates
!mkdir static
!cp gdrive/My\ Drive/StyleTransfer/index.html templates/
!cp gdrive/My\ Drive/StyleTransfer/generated.html templates/
!cp gdrive/My\ Drive/StyleTransfer/style.css static/
!cp gdrive/My\ Drive/StyleTransfer/function.js static/

In [ ]:
""" Get public url """
 
!pip install pyngrok
!ngrok authtoken xxxxx
from pyngrok import ngrok
 
public_url = ngrok.connect(port = '5000')
 
print(public_url)

ngrok.disconnect(public_url)

In [ ]:
""" Backend api """
 
from flask import Flask, request, render_template, url_for
from PIL import Image
 
app = Flask(__name__)
 
@app.route('/')
@app.route('/index')
def index():
 
    return render_template('index.html')
 
@app.route('/program', methods=['GET', 'POST'])
def program():
    if request.method == 'POST':
        if request.files:
 
            content = request.files.get('file1','')
            style = request.files.get('file2','')
            run(content, style)
 
    return render_template('index.html')
 
@app.route('/generated', methods=['GET', 'POST'])
def generated():
    return render_template('generated.html', content='/static/generated.jpg')
 
if __name__ == "__main__":
    app.run()

In [ ]:
ngrok.kill()